# Model Deployment

To deploy our collaborative-based model, we've created a web app using flask called 'Rate 5, Get 5'.  This allows movie lovers to rate 5 movies that they've seen and get back 5 movies that our model thinks they will also enjoy.  

Here, I provide the code for creating this app.  You can find full details, including css style sheets in the [`server`](../../server) folder of this repo. 

Two versions of this app are available:  
1. A demo app which randomly generates 5 movies for users to watch.  This is a light weight version that does not use modelling to generate the movies but is used to show how the app works in real time.  The model slows down the performance of the app and so we have a seperate app file for that. 
2. A model app which looks and works the same as the demo app expect the model is actually being used to generate the recommendations.  Note that running this on your computer will take a long time and so we recommend you take the results as found in the modelling section of this notebook.  

## Demp App Code

This file can be found in the server folder.  Run the file called app_demo.py in your terminal and copy the link into your browser to try out the app.  

In [ ]:
from flask import Flask, request, redirect, url_for
import pandas as pd

app = Flask(__name__)

movies = pd.read_csv('../data/mod_movies_lc', index_col=0)

@app.route('/', methods=['GET', 'POST'])
def get_home():    
    
    if len(request.form) == 5:
        return redirect(url_for('get_recs', data=request.form), code=307)
    
    body = """
    <html>
    <head>
        <title>Rate 5, Get 5</title>
        <link rel="stylesheet" type="text/css" href="/static/css/style.css">
    </head>
    <body>
    	<div class="container">
            <h1>Rate 5, Get 5</h1>
        	<form action="/" method="post">
        """
    
    # randomly select movies to rate
    movie_to_rate = movies.sample(1)
    
    if request.form is not None:
        for key in request.form.keys():
            if request.form[key] == "na":
                continue
            else:
                body += f"""
                <input type="hidden" name="{key}" value="{request.form[key]}">
                """
    
    # code for radio dials
    body +=  f"""
    			<div class="box-1">
	                <h2>Rate this Movie:</h2>
	                <div class="box-2">
			            <h3>{movie_to_rate['title'].values[0]}</h3>
			            	<ul>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_1" name="{movie_to_rate['movieId'].values[0]}" value="1">
				                <label for="{movie_to_rate['movieId']}_1">1</label><br></li>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_2" name="{movie_to_rate['movieId'].values[0]}" value="2">
				                <label for="{movie_to_rate['movieId']}_2">2</label><br></li>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_3" name="{movie_to_rate['movieId'].values[0]}" value="3">
				                <label for="{movie_to_rate['movieId']}_3">3</label><br></li>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_4" name="{movie_to_rate['movieId'].values[0]}" value="4">
				                <label for="{movie_to_rate['movieId']}_4">4</label><br></li>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_5" name="{movie_to_rate['movieId'].values[0]}" value="5">
				                <label for="{movie_to_rate['movieId']}_5">5</label><br></li>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_na" name="{movie_to_rate['movieId'].values[0]}" value="na">
				                <label for="{movie_to_rate['movieId']}_na">na</label><br></li>
				            </ul>
                            """
    
    # code to change button once user is making their 5th selection
    if len(request.form) == 4:
        body += """
		            	<input class="button" type="submit" value="Get My Recommendations!">
    """
    else:
        body += """
                        <input class="button" type="submit" value="Rate Another">
        """
    
    body += """
    </div>
                </div>      
            </form>
        </div>
    </body>
</html>
"""

    return body

@app.route("/recs", methods=['POST'])
def get_recs():

    body = f"""
    <html>
    <head>
        <title>Your Movie Recommendations</title>
        <link rel="stylesheet" type="text/css" href="/static/css/style_recs.css">
    </head>
    <body>
    	<div class="container">
    		<div class="box-1">
            	<h1>5 Movies, Just For You:</h1>
            </div>
    """      
    
    # for demo purposes, randomly select 5 movies to recommend
    recommend_5 = movies.sample(5)

    count = 1
    for movie in recommend_5.title:
        body += f"""
        <div class="box-2">
            	<h3>{count}. {movie}</h3>
            </div>
        """
        count += 1

    body += f"""
        </div>
    </body>
</html>  
    """
    
    return body


if __name__ == '__main__':
    app.run()

## Modelling App Code

This file can be found in the server folder.  Run the file called app.py in your terminal and copy the link into your browser to try out the app - this will take much longer to load after you have made your ratings. 

In [ ]:
from flask import Flask, request, redirect, url_for
import pandas as pd
import pickle
from surprise import Reader, Dataset
from surprise.prediction_algorithms import SVDpp

app = Flask(__name__)

movies = pd.read_csv('../data/mod_movies_lc', index_col=0)
ratings = pd.read_csv('../data/mod_ratings_lc', index_col=0)

# if using a pickled model, uncomment and add your file path
# model_file_path = 
# with open(model_file_path, 'rb') as f_in:
#     model = pickle.load(f_in)
#     f_in.close()
    
# MANUALLY ADD OUR BEST MODEL
model = SVDpp(n_factors=50, reg_all=0.05, lr_all=0.01)

@app.route('/', methods=['GET', 'POST'])
def get_home():    
    
    if len(request.form) == 5:
        return redirect(url_for('get_recs', data=request.form), code=307)
    
    body = """
    <html>
    <head>
        <title>Rate 5, Get 5</title>
        <link rel="stylesheet" type="text/css" href="/static/css/style.css">
    </head>
    <body>
    	<div class="container">
            <h1>Rate 5, Get 5</h1>
        	<form action="/" method="post">
        """
    
    movie_to_rate = movies.sample(1)
    
    if request.form is not None:
        for key in request.form.keys():
            if request.form[key] == "na":
                continue
            else:
                body += f"""
                <input type="hidden" name="{key}" value="{request.form[key]}">
                """
    
    body +=  f"""
    			<div class="box-1">
	                <h2>Rate this Movie:</h2>
	                <div class="box-2">
			            <h3>{movie_to_rate['title'].values[0]}</h3>
			            	<ul>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_1" name="{movie_to_rate['movieId'].values[0]}" value="1">
				                <label for="{movie_to_rate['movieId']}_1">1</label><br></li>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_2" name="{movie_to_rate['movieId'].values[0]}" value="2">
				                <label for="{movie_to_rate['movieId']}_2">2</label><br></li>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_3" name="{movie_to_rate['movieId'].values[0]}" value="3">
				                <label for="{movie_to_rate['movieId']}_3">3</label><br></li>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_4" name="{movie_to_rate['movieId'].values[0]}" value="4">
				                <label for="{movie_to_rate['movieId']}_4">4</label><br></li>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_5" name="{movie_to_rate['movieId'].values[0]}" value="5">
				                <label for="{movie_to_rate['movieId']}_5">5</label><br></li>
				                <li><input type="radio" id="{movie_to_rate['movieId']}_na" name="{movie_to_rate['movieId'].values[0]}" value="na">
				                <label for="{movie_to_rate['movieId']}_na">na</label><br></li>
				            </ul>
                            """
    
    if len(request.form) == 4:
        body += """
		            	<input class="button" type="submit" value="Get My Recommendations!">
    """
    else:
        body += """
                        <input class="button" type="submit" value="Rate Another">
        """
    
    body += """
    </div>
                </div>      
            </form>
        </div>
    </body>
</html>
"""
    return body


@app.route("/recs", methods=['POST'])
def get_recs():

    body = f"""
    <html>
    <head>
        <title>Your Movie Recommendations</title>
        <link rel="stylesheet" type="text/css" href="/static/css/style_recs.css">
    </head>
    <body>
    	<div class="container">
    		<div class="box-1">
            	<h1>5 Movies, Just For You:</h1>
            </div>
    """      
    userId = 1000
    user_ratings = []
    reader = Reader()
    
    # create list of dictionary items containing user's ratings from previous page
    for movieId in request.form.keys():
        rating_one_movie = {'userId': userId, 'movieId': movieId, 'rating': int(request.form[movieId])}
        user_ratings.append(rating_one_movie)
        
    
    # add the new ratings to the original movies df
    combined_ratings = ratings.append(user_ratings, ignore_index=True)
    data = Dataset.load_from_df(combined_ratings, reader)
    
    # fit the model to the data with the new user's ratings
    model.fit(data.build_full_trainset())
    
    # create a list of tuples containing the new user's predicted ratings for each movie
    list_of_movies = []
    for movieId in ratings['movieId'].unique():
        print(model.predict(userId, movieId)[3])
        list_of_movies.append((movieId, model.predict(userId, movieId)[3]))
    
    # rank the predictions from highest to lowest
    rank_movies = sorted(list_of_movies, key = lambda x:x[1], reverse=True)
    
    # grab the top 5 highest predicted ratings
    top_5 = rank_movies[:5]
    
    # print out the top 5 recommendations
    count = 1
    for movie in top_5:
        body += f"""
        <div class="box-2">
            	<h3>{count}. {movies[movies['movieId'] == movie[0]]['title'].values[0]}</h3>
            </div>
        """
        count += 1

    body += f"""
        </div>
    </body>
</html>  
    """  
    return body


if __name__ == '__main__':
    app.run()

## App Preview

Here's a preview of what the app looks like:

#### Home Page:
![image](../../reports/figures/app_home.png)

#### Recommendations Page:
![image](../../reports/figures/app_recs.png)